# Objective:: Classify texts in 7 different discrete emotions - (anger, disgust, fear, joy, surprise and neutral using LLM)

In [1]:
# import libraries
import pandas as pd
import numpy as np
from transformers import pipeline

In [2]:
# load the data
df = pd.read_csv('books_cleaned.csv')
df.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_subtitle,tagged_desc
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web:A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."


In [5]:
df.columns, df.shape

(Index(['isbn13', 'isbn10', 'title', 'authors', 'categories', 'thumbnail',
        'description', 'published_year', 'average_rating', 'num_pages',
        'ratings_count', 'title_subtitle', 'tagged_desc'],
       dtype='object'),
 (5197, 13))

In [ ]:
# to reduce computing time, we wil select just a 1000 records
df = df[['isbn13','categories','description', 'title_subtitle', 'tagged_desc']]
df.sample(n=1000, random_state=42)
df.head()

,isbn13,categories,description,title_subtitle,tagged_desc
0,9780002005883,Fiction,A NOVEL THAT READERS and critics have been eag...,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,Detective and mystery stories,A new 'Christie for Christmas' -- a full-lengt...,Spider's Web:A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,Fiction,"A memorable, mesmerizing heroine Jennifer -- b...",Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,Christian life,Lewis' work on the nature of love divides love...,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,Christian life,"""In The Problem of Pain, C.S. Lewis, one of th...",The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."


In [7]:
# load a fine-tuned llm text classification model
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/329M [00:00<?, ?B/s]

2025-03-13 18:42:44.909448: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-03-13 18:42:44.910456: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-03-13 18:42:44.910484: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
I0000 00:00:1741891364.911561  992471 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1741891364.911838  992471 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at j-hartmann/emotion-english-distilroberta-base.
If your task is similar to 

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use 0
/Users/chibuikeiwuchukwu/Docs/ML_Project/nlp_projects/nlp-env/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
# examine a sample sequence - since the first description is quite long we can split it to enabl the model predict an accurate
# emotion
sequence = df['description'][0].split('.')

In [12]:
sequence

['A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives',
 ' John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers',
 ' It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up',
 ' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist',
 ' He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption',
 ' Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and t

In [11]:
result = classifier(sequence)
result

[[{'label': 'anger', 'score': 0.009156346321105957},
  {'label': 'disgust', 'score': 0.0026284735649824142},
  {'label': 'fear', 'score': 0.06816209107637405},
  {'label': 'joy', 'score': 0.04794241860508919},
  {'label': 'neutral', 'score': 0.140385240316391},
  {'label': 'sadness', 'score': 0.0021221591159701347},
  {'label': 'surprise', 'score': 0.7296032905578613}],
 [{'label': 'anger', 'score': 0.040478236973285675},
  {'label': 'disgust', 'score': 0.27359116077423096},
  {'label': 'fear', 'score': 0.006879059132188559},
  {'label': 'joy', 'score': 0.10908257216215134},
  {'label': 'neutral', 'score': 0.44937148690223694},
  {'label': 'sadness', 'score': 0.09362729638814926},
  {'label': 'surprise', 'score': 0.026970157399773598}],
 [{'label': 'anger', 'score': 0.011031879112124443},
  {'label': 'disgust', 'score': 0.043422598391771317},
  {'label': 'fear', 'score': 0.014084081165492535},
  {'label': 'joy', 'score': 0.014211437664926052},
  {'label': 'neutral', 'score': 0.64621585

In [64]:
def compute_the_max_emotion_scores(predictions):

    emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
    per_emotion_score = {label: [] for label in emotion_labels}

    for results in predictions:
        sorted_predictions =  sorted(results, key= lambda x: x['label'])
        for index, label in enumerate(emotion_labels):
            per_emotion_score[label].append(sorted_predictions[index]['score'])

    return {label: np.max(scores) for label, scores in per_emotion_score.items()}

In [65]:
emotions_score_dict = compute_the_max_emotion_scores(result)
emotions_score_dict

{'anger': 0.06413356214761734,
 'disgust': 0.27359116077423096,
 'fear': 0.928168535232544,
 'joy': 0.9327978491783142,
 'neutral': 0.6462158560752869,
 'sadness': 0.9671575427055359,
 'surprise': 0.7296032905578613}

In [66]:
from tqdm import tqdm

In [69]:
emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
emotion_scores = {label: [] for label in emotion_labels}

# selected a few records and return their emotional sentiments
num_of_records = 10
isbn = []
for _ in tqdm(range(num_of_records)):
    isbn.append(df['isbn13'][_])
    sentence = df['description'][_].split('.')
    pred = classifier(sentence)
    scores = compute_the_max_emotion_scores(pred)
    for label in emotion_labels:
        emotion_scores[label].append(scores[label])

100%|██████████| 10/10 [00:08<00:00,  1.21it/s]


In [70]:
emotion_scores

{'anger': [0.06413356214761734,
  0.612618088722229,
  0.06413356214761734,
  0.35148385167121887,
  0.08141220360994339,
  0.23222456872463226,
  0.5381852984428406,
  0.06413356214761734,
  0.30067068338394165,
  0.06413356214761734],
 'disgust': [0.27359116077423096,
  0.34828609228134155,
  0.1040065810084343,
  0.1507224142551422,
  0.18449489772319794,
  0.7271750569343567,
  0.15585501492023468,
  0.1040065810084343,
  0.2794811725616455,
  0.1779274046421051],
 'fear': [0.928168535232544,
  0.942527711391449,
  0.9723208546638489,
  0.3607065975666046,
  0.09504332393407822,
  0.05136275663971901,
  0.7474286556243896,
  0.404496967792511,
  0.9155242443084717,
  0.05136275663971901],
 'joy': [0.9327978491783142,
  0.7044218182563782,
  0.7672367691993713,
  0.2518808841705322,
  0.04056438058614731,
  0.04337581619620323,
  0.8725653886795044,
  0.04056438058614731,
  0.04056438058614731,
  0.04056438058614731],
 'neutral': [0.6462158560752869,
  0.8879395127296448,
  0.549477

In [71]:
# creating a dataframe
emotions_df = pd.DataFrame(emotion_scores)
emotions_df['isbn13'] =  isbn
emotions_df.head()

,anger,disgust,fear,joy,neutral,sadness,surprise,isbn13
0,0.064134,0.273591,0.928169,0.932798,0.646216,0.967158,0.729603,9780002005883
1,0.612618,0.348286,0.942528,0.704422,0.887940,0.111690,0.252545,9780002261982
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765,9780006178736
3,0.351484,0.150722,0.360707,0.251881,0.732686,0.111690,0.078765,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078765,9780006280934


In [72]:
# merging the emotions_df to the original df
df = pd.merge(df, emotions_df, on='isbn13')
df.head()

,isbn13,categories,description,title_subtitle,tagged_desc,anger,disgust,fear,joy,neutral,sadness,surprise
0,9780002005883,Fiction,A NOVEL THAT READERS and critics have been eag...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,0.064134,0.273591,0.928169,0.932798,0.646216,0.967158,0.729603
1,9780002261982,Detective and mystery stories,A new 'Christie for Christmas' -- a full-lengt...,Spider's Web:A Novel,9780002261982 A new 'Christie for Christmas' -...,0.612618,0.348286,0.942528,0.704422,0.887940,0.111690,0.252545
2,9780006178736,Fiction,"A memorable, mesmerizing heroine Jennifer -- b...",Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765
3,9780006280897,Christian life,Lewis' work on the nature of love divides love...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,0.351484,0.150722,0.360707,0.251881,0.732686,0.111690,0.078765
4,9780006280934,Christian life,"""In The Problem of Pain, C.S. Lewis, one of th...",The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078765
